In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.preprocessing import LabelEncoder

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using CUDA: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

C:\Users\Arthur\OneDrive - Vrije Universiteit Brussel\Master jaar\Masterproef\PY scripts\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'AdamW' from 'transformers' (C:\Users\Arthur\OneDrive - Vrije Universiteit Brussel\Master jaar\Masterproef\PY scripts\.venv\lib\site-packages\transformers\__init__.py)

In [ ]:
# Define BERT model and tokenizer
MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2) # 2 labels: non-fraud (0) and fraud (1)
model.to(device)

# Define hyperparameters
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 3